In [17]:
import cv2
import numpy as np
import easyocr
import imutils
import serial
import time
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import os
import json
import serial.tools.list_ports

spz_file = "spravne_spz.json"
history_file = "spz_history.json"

class SPZApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Aplikace pro rozpoznávání SPZ")
        self.root.geometry("600x600")  # Increased height to fit the new button
        self.center_window()
        self.root.resizable(False, False)

        self.spravne_spz_list = self.load_spravne_spz()
        self.spz_history = self.load_spz_history()

        self.title_label = tk.Label(root, text="Aplikace pro rozpoznávání SPZ", font=("Arial", 14, "bold"))
        self.title_label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.place(relx=0.2, rely=0.4, relwidth=0.6, relheight=0.4)

        self.result_label = tk.Label(root, text="", font=("Arial", 12))
        self.result_label.pack(pady=5)

        self.btn_frame = tk.Frame(root)
        self.btn_frame.pack(pady=10)

        tk.Button(self.btn_frame, text="Vyber obrázek", command=self.process_file_image).grid(row=0, column=0, padx=5)
        tk.Button(self.btn_frame, text="Použij webkameru", command=self.process_camera_image).grid(row=0, column=1, padx=5)
        tk.Button(self.btn_frame, text="Náhled webkamery", command=self.toggle_camera_preview).grid(row=0, column=2, padx=5)

        tk.Button(root, text="Zobraz SPZ seznam", command=self.show_spz_list).pack(pady=5)
        
        # Button for showing SPZ history
        tk.Button(root, text="Zobraz Historii SPZ", command=self.show_spz_history).pack(pady=5)

        self.edit_frame = tk.Frame(root)
        self.edit_frame.pack(pady=5)

        tk.Button(self.edit_frame, text="Přidat SPZ", command=self.add_spz).grid(row=0, column=0, padx=5)
        tk.Button(self.edit_frame, text="Odebrat SPZ (index)", command=self.remove_spz).grid(row=0, column=1, padx=5)

        self.cap = None
        self.previewing = False

        # Automatická detekce Arduino portu
        arduino_port = self.find_arduino_port()
        if arduino_port:
            try:
                self.arduino = serial.Serial(arduino_port, 9600, timeout=1)
                time.sleep(2)
            except serial.SerialException:
                self.arduino = None
                messagebox.showwarning("Upozornění", f"Port {arduino_port} nalezen, ale nepodařilo se připojit k Arduinu.")
        else:
            self.arduino = None
            messagebox.showwarning("Upozornění", "Nepodařilo se automaticky najít Arduino port.")

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def find_arduino_port(self):
        """Zkusí otevřít každý dostupný port a zjistit, zda na něm je Arduino nebo jeho klon."""
        ports = serial.tools.list_ports.comports()
        for port in ports:
            try:
                # Pokusíme se otevřít port
                test_serial = serial.Serial(port.device, 9600, timeout=1)
                test_serial.close()  # Uzavřeme port, pokud je dostupný
                return port.device  # Pokud se port otevřel, je to správný port
            except (serial.SerialException, OSError):
                continue  # Pokud port nelze otevřít, zkusíme další

        return None  # Pokud nenalezneme žádný otevřitelný port, vrátíme None

    def center_window(self):
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'{width}x{height}+{x}+{y}')

    def process_file_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Obrázky", "*.jpg *.jpeg *.png")])
        if file_path:
            img = cv2.imread(file_path)
            self.show_image(img)
            self.detect_spz(img)

    def process_camera_image(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Chyba", "Nelze otevřít webkameru.")
            return
        ret, frame = cap.read()
        cap.release()
        if not ret:
            messagebox.showerror("Chyba", "Nepodařilo se načíst snímek z kamery.")
        else:
            self.show_image(frame)
            self.detect_spz(frame)

    def toggle_camera_preview(self):
        if not self.previewing:
            self.previewing = True
            self.start_camera_preview()
        else:
            self.previewing = False
            self.stop_camera_preview()

    def start_camera_preview(self):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Chyba", "Nelze otevřít webkameru.")
            return
        self.update_preview()

    def stop_camera_preview(self):
        if self.cap and self.cap.isOpened():
            self.cap.release()
        self.image_label.config(image="")
        self.result_label.config(text="")
        self.previewing = False

    def update_preview(self):
        if self.cap and self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                return
            self.show_image(frame)
            if self.previewing:
                self.root.after(10, self.update_preview)

    def show_image(self, img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb).resize((300, 200))
        img_tk = ImageTk.PhotoImage(image=img_pil)
        self.image_label.imgtk = img_tk
        self.image_label.config(image=img_tk)

    def detect_spz(self, img):
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
            edged = cv2.Canny(bfilter, 30, 200)
            keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(keypoints)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

            location = None
            for contour in contours:
                approx = cv2.approxPolyDP(contour, 10, True)
                if len(approx) == 4:
                    location = approx
                    break

            if location is None:
                raise ValueError("Nepodařilo se najít SPZ.")

            mask = np.zeros(gray.shape, np.uint8)
            cv2.drawContours(mask, [location], 0, 255, -1)
            (x, y) = np.where(mask == 255)
            cropped_image = gray[np.min(x):np.max(x)+1, np.min(y):np.max(y)+1]

            reader = easyocr.Reader(['en'], gpu=False)
            result = reader.readtext(cropped_image)
            if not result:
                raise ValueError("OCR nerozpoznalo žádný text.")
            text = result[0][-2].replace(" ", "")
            print("Rozpoznaná SPZ:", text)

            # Přidání rozpoznaného textu na obrázek
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_position = (10, img.shape[0] - 10)  # Pozice textu v dolní části obrázku
            cv2.putText(img, f"SPZ: {text}", text_position, font, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Zobrazení textu v GUI
            if text in self.spravne_spz_list:
                self.result_label.config(text=f"✅ SPZ '{text}' je správná!", fg="green")
                if self.arduino:
                    self.arduino.write(b'1')
            else:
                self.result_label.config(text=f"❌ SPZ '{text}' není v seznamu.", fg="red")
                if self.arduino:
                    self.arduino.write(b'0')

            # Uložení SPZ do historie
            self.update_spz_history(text)

            # Zobrazí obrázek s přidaným textem
            self.show_image(img)

        except Exception as e:
            messagebox.showerror("Chyba", str(e))

    def load_spravne_spz(self):
        if os.path.exists(spz_file):
            with open(spz_file, "r") as f:
                return json.load(f)
        return []

    def save_spravne_spz(self):
        with open(spz_file, "w") as f:
            json.dump(self.spravne_spz_list, f)

    def load_spz_history(self):
        if os.path.exists(history_file):
            with open(history_file, "r") as f:
                return json.load(f)
        return []

    def save_spz_history(self):
        with open(history_file, "w") as f:
            json.dump(self.spz_history, f)

    def update_spz_history(self, spz):
        if spz not in [entry['spz'] for entry in self.spz_history]:
            self.spz_history.append({'spz': spz, 'star': '*' if spz in self.spravne_spz_list else ''})
            self.save_spz_history()

    def show_spz_list(self):
        if not self.spravne_spz_list:
            messagebox.showinfo("SPZ seznam", "Seznam je prázdný.")
        else:
            seznam = "\n".join(f"{i+1}. {spz}" for i, spz in enumerate(self.spravne_spz_list))
            messagebox.showinfo("SPZ seznam", seznam)

    def show_spz_history(self):
        if not self.spz_history:
            messagebox.showinfo("Historie SPZ", "Historie je prázdná.")
        else:
            history_text = "\n".join(f"{i+1}. {entry['spz']} {entry['star']}" for i, entry in enumerate(self.spz_history))
            messagebox.showinfo("Historie SPZ", history_text)

    def add_spz(self):
        new_spz = simpledialog.askstring("Přidat SPZ", "Zadejte novou SPZ:")
        if new_spz:
            self.spravne_spz_list.append(new_spz.strip())
            self.save_spravne_spz()
            messagebox.showinfo("Hotovo", f"SPZ '{new_spz}' přidána.")

    def remove_spz(self):
        index_str = simpledialog.askstring("Odebrat SPZ", "Zadejte index SPZ (1-n):")
        if index_str and index_str.isdigit():
            index = int(index_str) - 1
            if 0 <= index < len(self.spravne_spz_list):
                removed = self.spravne_spz_list.pop(index)
                self.save_spravne_spz()
                messagebox.showinfo("Odebráno", f"SPZ '{removed}' byla odebrána.")
            else:
                messagebox.showerror("Chyba", "Neplatný index.")

    def on_close(self):
        if self.cap and self.cap.isOpened():
            self.cap.release()
        if self.arduino and self.arduino.is_open:
            self.arduino.close()
        self.save_spravne_spz()
        self.save_spz_history()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = SPZApp(root)
    root.mainloop()
